# MScA, Advanced Machine Learning (32017)

# Week 1 Assignment: Introduction to Spark

## Installation of Spark on local computer

Follow instructions in the <a href="https://ilykei.com/api/fileProxy/documents%2FAdvanced%20Machine%20Learning%2FLecture%201%20AdvML%2FSparkInstallation0.1.pdf" target="_blank"> document prepared by Ashish Pujari</a>

Check that installation is working by running test code at the end of the installation instructions notebook.

## Example

Run the following example to start using Spark from your notebook.

See if you need to refresh your knowledge of lambda functions in python or other basic python skills.
If some pyspark commands (like, for example, `.map()`, `flatMap()`, etc.) do not make sense - no need to worry: we will go over them next week. 


Create an RDD from the first 5 lines of Shakespeare's "A Midsummer-Night's Dream". <br>
The result is a list of elements.

In [9]:
txtRDD=sc.textFile("./Examples/shakespeare.txt")
txtRDD.take(5)

["A MIDSUMMER-NIGHT'S DREAM",
 '',
 'Now , fair Hippolyta , our nuptial hour ',
 'Draws on apace : four happy days bring in ',
 'Another moon ; but O ! methinks how slow ']

After executing the cell above find your localhost UI by typing `localhost:4040` in your browser.
If you experience issues connecting look at diagnostics in your command window or terminal: it is possible that localhost port is replaced by, for example 4041.

On Jobs tab find the job executed by the cell above. 
On Stages tab click the stage link.
-  Find DAG visualization for the job;
-  Find duration;
-  Find Input Size / Records;
-  Check the number of data partitions (RDD Blocks) for this job

Repeat the same job with number of partitions set.

In [10]:
txtRDD=sc.textFile("./Examples/shakespeare.txt",minPartitions=4)
txtRDD.take(5)

["A MIDSUMMER-NIGHT'S DREAM",
 '',
 'Now , fair Hippolyta , our nuptial hour ',
 'Draws on apace : four happy days bring in ',
 'Another moon ; but O ! methinks how slow ']

Check the new job which appeared in Driver UI.
What has changed?

Summarize the information collected from UI in the following empty cell.

Summary of information from Driver UI:


Apply one step of type Map: split the data into separate words using `.map('\W+')` which returns a new RDD with 
lambda function applied to each of the elements of the original list. The output is in the form of list of lists.

In [11]:
import re
txtRDD.map(lambda line: re.split('\W+',line)).take(5)

[['A', 'MIDSUMMER', 'NIGHT', 'S', 'DREAM'],
 [''],
 ['Now', 'fair', 'Hippolyta', 'our', 'nuptial', 'hour', ''],
 ['Draws', 'on', 'apace', 'four', 'happy', 'days', 'bring', 'in', ''],
 ['Another', 'moon', 'but', 'O', 'methinks', 'how', 'slow', '']]

Now do the same, but using `.flatMap()` which returns new RDD as single list of elements, rather than list of lists. 

In [12]:
txtRDD.flatMap(lambda line: re.split('\W+',line)).take(10)

['A',
 'MIDSUMMER',
 'NIGHT',
 'S',
 'DREAM',
 '',
 'Now',
 'fair',
 'Hippolyta',
 'our']

Now apply 2 Map steps: <br>
First `.flatMap()` to separate words. <br> 
Then `.map()` to prepare key-value pair needed for counting: (key,1); here key will be a word from Shakespeare.

In [13]:
kvrdd=txtRDD \
.flatMap(lambda line: re.split('\W+',line)) \
.map(lambda word: (word,1))
kvrdd.take(10)

[('A', 1),
 ('MIDSUMMER', 1),
 ('NIGHT', 1),
 ('S', 1),
 ('DREAM', 1),
 ('', 1),
 ('Now', 1),
 ('fair', 1),
 ('Hippolyta', 1),
 ('our', 1)]

The elements of the output list are tuples of the form `(k,v)`. <br>
<br>
After these two Map steps use `reduceByKey()` to summarize total by key. <br> 
Result is another list of tuples.

In [14]:
kvrdd.reduceByKey(lambda v1,v2: v1+v2).take(10)

[('NIGHT', 2),
 ('', 130028),
 ('apace', 25),
 ('days', 198),
 ('in', 9584),
 ('Another', 23),
 ('but', 3711),
 ('O', 2299),
 ('lingers', 4),
 ('step', 29)]